# Import the relevant packages

In [1]:
import numpy as np 
import tensorflow as tf
import tensorflow_datasets as tfds 

# Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = tf.cast(0.1 * mnist_info.splits['train'].num_examples, tf.int64)

num_test_samples = tf.cast(0.1 * mnist_info.splits['test'].num_examples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

buffer_size = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

batch_size = 100
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Model

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape = (28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'),
                            tf.keras.layers.Dense(output_size, activation = 'softmax')
                            ])

# Choose the optimizer and the loss function

In [4]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
tf.keras.optimizers.Adam(learning_rate=0.0001)

# Training

In [5]:
num_epochs = 10

model.fit(train_data, epochs = num_epochs, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/10
540/540 - 3s - loss: 0.2502 - accuracy: 0.9247 - val_loss: 0.1127 - val_accuracy: 0.9665
Epoch 2/10
540/540 - 3s - loss: 0.0955 - accuracy: 0.9699 - val_loss: 0.0792 - val_accuracy: 0.9758
Epoch 3/10
540/540 - 3s - loss: 0.0679 - accuracy: 0.9782 - val_loss: 0.0601 - val_accuracy: 0.9808
Epoch 4/10
540/540 - 3s - loss: 0.0473 - accuracy: 0.9847 - val_loss: 0.0444 - val_accuracy: 0.9867
Epoch 5/10
540/540 - 3s - loss: 0.0380 - accuracy: 0.9878 - val_loss: 0.0539 - val_accuracy: 0.9840
Epoch 6/10
540/540 - 3s - loss: 0.0319 - accuracy: 0.9893 - val_loss: 0.0439 - val_accuracy: 0.9857
Epoch 7/10
540/540 - 3s - loss: 0.0261 - accuracy: 0.9914 - val_loss: 0.0281 - val_accuracy: 0.9903
Epoch 8/10
540/540 - 3s - loss: 0.0232 - accuracy: 0.9924 - val_loss: 0.0352 - val_accuracy: 0.9898
Epoch 9/10
540/540 - 3s - loss: 0.0201 - accuracy: 0.9932 - val_loss: 0.0216 - val_accuracy: 0.9930
Epoch 10/10
540/540 - 3s - loss: 0.0212 - accuracy: 0.9931 - val_loss: 0.0142 - val_accuracy: 0.9953

# Test the model

In [8]:
test_loss, test_accuracy = model.evaluate(test_data)
print("Test Loss: {0:.2f}. Test Accuracy: {1:.2f}%".format(test_loss,test_accuracy*100.))

10/10 [==============================] - 1s 68ms/step - loss: 0.0650 - accuracy: 0.9831
Test Loss: 0.06. Test Accuracy: 98.31%
